In [1]:
import numpy as np
import random
seed = 666
random.seed(seed)
np.random.seed(seed)

import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score

from clusterless import data_preprocess
from clusterless import viz_tools
from clusterless import init_gmm
from clusterless import split_merge
from clusterless import smooth_signals
from clusterless import decode_static

In [2]:
rootpath = "data"
sub_id = "dy016"
samp_freq = 30_000

In [75]:
# load behavior data
behave_dict, behave_idx_dict = data_preprocess.load_behaviors_data(rootpath, sub_id)

# preprocess behavior data
choices, stimuli, transformed_stimuli, one_hot_stimuli, rewards, priors = data_preprocess.preprocess_static_behaviors(behave_dict)
static_behaviors = {'choice': choices, 'stimulus': one_hot_stimuli}

choices left: 0.769, right: 0.231
stimuli left: 0.507, right: 0.493
reward wrong: 0.231, correct: 0.769


all

In [101]:
# load sorted data
ks_spikes_times, ks_spikes_clusters, ks_spikes_amps, ks_spikes_depths, sorted, ks_trials = \
    data_preprocess.load_kilosort_sorted_data(rootpath, sub_id, keep_active_trials=False, samp_freq=samp_freq)

# load kilosort units that passed ibl's quality metrics
good_ks_spikes_indices = data_preprocess.load_kilosort_good_ibl_units(rootpath, sub_id, roi='all', keep_active_trials = False, samp_freq=30_000)

In [103]:
# load unsorted data
spikes_indices, spikes_features, np1_channel_map, stimulus_onset_times, unsorted, trials = \
    data_preprocess.load_unsorted_data(rootpath, sub_id, roi='all', keep_active_trials=False, samp_freq=samp_freq)

# load clusterless model
trials_ids = np.arange(len(trials))
spikes_times = np.vstack([trials[i] for i in trials_ids])[:,0]
structured_unsorted = np.vstack([trials[i] for i in trials_ids])[:,1:] # follows time order

# load gmm after split-merge
gmm = split_merge.merge_gaussians('.', sub_id, structured_unsorted, post_split_gmm=None, post_split_labels=None, merge_ids=None, fit_model=False)
labels = gmm.predict(structured_unsorted)
probs = gmm.predict_proba(structured_unsorted)

In [ ]:
# prepare time binned neural activity for decoding
n_projs = 10
neural_data_dict = dict()
tpca_recon_dict = dict()

sorted_neural_data = data_preprocess.compute_time_binned_neural_activity((ks_spikes_times, ks_spikes_clusters), 'sorted', stimulus_onset_times, regional=False)
sorted_tpca_proj, sorted_tpca_recon = smooth_signals.tpca(sorted_neural_data, n_projs=n_projs)
neural_data_dict.update({'sorted': sorted_neural_data})
tpca_recon_dict.update({'sorted': sorted_tpca_recon})

good_ks_neural_data = data_preprocess.compute_time_binned_neural_activity(good_ks_spikes_indices, 'good units', stimulus_onset_times, regional=False)
good_ks_tpca_proj, good_ks_tpca_recon = smooth_signals.tpca(good_ks_neural_data, n_projs=n_projs)
neural_data_dict.update({'good units': good_ks_neural_data})
tpca_recon_dict.update({'good units': good_ks_tpca_recon})

unsorted_neural_data = data_preprocess.compute_time_binned_neural_activity(spikes_indices, 'unsorted', stimulus_onset_times, regional=False)
unsorted_tpca_proj, unsorted_tpca_recon = smooth_signals.tpca(unsorted_neural_data, n_projs=n_projs)
neural_data_dict.update({'unsorted': unsorted_neural_data})
tpca_recon_dict.update({'unsorted': unsorted_tpca_recon})

clusterless_neural_data = data_preprocess.\
    compute_time_binned_neural_activity((spikes_times, labels, probs), 
                                            'clusterless', stimulus_onset_times, regional=False)
clusterless_tpca_proj, clusterless_tpca_recon = smooth_signals.tpca(clusterless_neural_data, n_projs=n_projs)
neural_data_dict.update({'clusterless': clusterless_neural_data})
tpca_recon_dict.update({'clusterless': clusterless_tpca_recon})

In [ ]:
# decode using original neural activity
all_decode_results = {'choice': dict(), 'stimulus': dict()}

for behave_type in all_decode_results.keys():
    for data_type in neural_data_dict.keys():
        print(f'{data_type} decoding ...')
        decode_results = decode_static.cv_decode_static(neural_data_dict[data_type], static_behaviors[behave_type], behave_type=behave_type, n_folds=5, seed=seed, shuffle=True)
        all_decode_results[behave_type].update({data_type: decode_results})
np.save(f'{rootpath}/{sub_id}/decode_results/all_decode_results', all_decode_results)

In [ ]:
# decode using smoothed neural activity by tpca
all_tpca_decode_results = {'choice': dict(), 'stimulus': dict()}

for behave_type in all_tpca_decode_results.keys():
    for data_type in tpca_recon_dict.keys():
        print(f'{data_type} decoding ...')
        decode_results = decode_static.cv_decode_static(tpca_recon_dict[data_type], static_behaviors[behave_type], behave_type=behave_type, n_folds=5, seed=seed, shuffle=True)
        all_tpca_decode_results[behave_type].update({data_type: decode_results})
np.save(f'{rootpath}/{sub_id}/decode_results/all_tpca_decode_results', all_tpca_decode_results)

In [63]:
# load decoding results for plotting
all_decode_results = np.load(f'{rootpath}/{sub_id}/decode_results/all_decode_results.npy', allow_pickle=True).item()
all_tpca_decode_results = np.load(f'{rootpath}/{sub_id}/decode_results/all_tpca_decode_results.npy', allow_pickle=True).item()

regional

In [87]:
rois = ['PO', 'LP', 'DG', 'CA1', 'PPC']
roi = rois[0]

In [95]:
# load sorted data
regional_sorted = data_preprocess.load_kilosort_sorted_data(rootpath, sub_id, roi=roi, keep_active_trials = False, samp_freq=30_000)

# load kilosort units that passed ibl's quality metrics
regional_good_spikes_indices = data_preprocess.load_kilosort_good_ibl_units(rootpath, sub_id, roi=roi, keep_active_trials = False, samp_freq=30_000)

In [89]:
# load unsorted datas
regional_trials = data_preprocess.load_unsorted_data(rootpath, sub_id, roi=roi, keep_active_trials=False, samp_freq=30_000)
regional_spikes_times = np.vstack([regional_trials[i] for i in range(len(regional_trials))])[:,0]
regional_unsorted = np.vstack([regional_trials[i] for i in range(len(regional_trials))])[:,:2]

# load clusterless data
regional_clusterless = np.vstack([regional_trials[i] for i in range(len(regional_trials))])[:,2:]

# load gmm after split-merge
gmm = split_merge.merge_gaussians('.', sub_id, structured_unsorted, post_split_gmm=None, post_split_labels=None, merge_ids=None, fit_model=False)
regional_labels = gmm.predict(regional_clusterless)
regional_probs = gmm.predict_proba(regional_clusterless)

found 58 channels in roi PO


In [96]:
# prepare time binned neural activity for decoding
n_projs = 10
neural_data_dict = dict()
tpca_recon_dict = dict()

regional_sorted_neural_data = data_preprocess.compute_time_binned_neural_activity(regional_sorted, 'sorted', stimulus_onset_times, regional=True)
regional_sorted_tpca_proj, regional_sorted_tpca_recon = smooth_signals.tpca(regional_sorted_neural_data, n_projs=n_projs)
neural_data_dict.update({'sorted': regional_sorted_neural_data})
tpca_recon_dict.update({'sorted': regional_sorted_tpca_recon})

regional_good_neural_data = data_preprocess.compute_time_binned_neural_activity(regional_good_spikes_indices, 'good units', stimulus_onset_times, regional=True)
regional_good_tpca_proj, regional_good_tpca_recon = smooth_signals.tpca(regional_good_neural_data, n_projs=n_projs)
neural_data_dict.update({'good units': regional_good_neural_data})
tpca_recon_dict.update({'good units': regional_good_tpca_recon})

regional_unsorted_neural_data = data_preprocess.compute_time_binned_neural_activity(regional_unsorted, 'unsorted', stimulus_onset_times, regional=True)
regional_unsorted_tpca_proj, regional_unsorted_tpca_recon = smooth_signals.tpca(regional_unsorted_neural_data, n_projs=n_projs)
neural_data_dict.update({'unsorted': regional_unsorted_neural_data})
tpca_recon_dict.update({'unsorted': regional_unsorted_tpca_recon})

regional_clusterless_neural_data = data_preprocess.\
    compute_time_binned_neural_activity((regional_spikes_times, regional_labels, regional_probs), 
                                            'clusterless', stimulus_onset_times, regional=True)
regional_clusterless_tpca_proj, regional_clusterless_tpca_recon = smooth_signals.tpca(regional_clusterless_neural_data, n_projs=n_projs)
neural_data_dict.update({'clusterless': regional_clusterless_neural_data})
tpca_recon_dict.update({'clusterless': regional_clusterless_tpca_recon})

In [76]:
# decode using original neural activity
regional_decode_results = {'choice': dict(), 'stimulus': dict()}

for behave_type in regional_decode_results.keys():
    for data_type in neural_data_dict.keys():
        print(f'{data_type} decoding ...')
        decode_results = decode_static.cv_decode_static(neural_data_dict[data_type], static_behaviors[behave_type], behave_type=behave_type, n_folds=5, seed=seed, shuffle=True)
        regional_decode_results[behave_type].update({data_type: decode_results})
np.save(f'{rootpath}/{sub_id}/decode_results/{roi}_decode_results', regional_decode_results)

sorted decoding ...
choice fold 1 test accuracy: 0.879 auc: 0.945
choice fold 2 test accuracy: 0.807 auc: 0.878
choice fold 3 test accuracy: 0.895 auc: 0.883
choice fold 4 test accuracy: 0.860 auc: 0.917
choice fold 5 test accuracy: 0.842 auc: 0.830
choice mean of 5-fold cv accuracy: 0.857 auc: 0.891
choice sd of 5-fold cv accuracy: 0.030 auc: 0.039
good units decoding ...
choice fold 1 test accuracy: 0.897 auc: 0.904
choice fold 2 test accuracy: 0.807 auc: 0.886
choice fold 3 test accuracy: 0.842 auc: 0.908
choice fold 4 test accuracy: 0.930 auc: 0.960
choice fold 5 test accuracy: 0.877 auc: 0.853
choice mean of 5-fold cv accuracy: 0.871 auc: 0.902
choice sd of 5-fold cv accuracy: 0.043 auc: 0.035
unsorted decoding ...
choice fold 1 test accuracy: 0.879 auc: 0.932
choice fold 2 test accuracy: 0.789 auc: 0.893
choice fold 3 test accuracy: 0.947 auc: 0.806
choice fold 4 test accuracy: 0.825 auc: 0.900
choice fold 5 test accuracy: 0.825 auc: 0.847
choice mean of 5-fold cv accuracy: 0.853

In [98]:
# decode using smoothed neural activity by tpca
regional_tpca_decode_results = {'choice': dict(), 'stimulus': dict()}

for behave_type in all_tpca_decode_results.keys():
    for data_type in tpca_recon_dict.keys():
        print(f'{data_type} decoding ...')
        decode_results = decode_static.cv_decode_static(tpca_recon_dict[data_type], static_behaviors[behave_type], behave_type=behave_type, n_folds=5, seed=seed, shuffle=True)
        regional_tpca_decode_results[behave_type].update({data_type: decode_results})
np.save(f'{rootpath}/{sub_id}/decode_results/{roi}_tpca_decode_results', regional_tpca_decode_results)

sorted decoding ...
choice fold 1 test accuracy: 0.914 auc: 0.950
choice fold 2 test accuracy: 0.807 auc: 0.897
choice fold 3 test accuracy: 0.912 auc: 0.832
choice fold 4 test accuracy: 0.825 auc: 0.935
choice fold 5 test accuracy: 0.842 auc: 0.819
choice mean of 5-fold cv accuracy: 0.860 auc: 0.887
choice sd of 5-fold cv accuracy: 0.045 auc: 0.053
good units decoding ...
choice fold 1 test accuracy: 0.828 auc: 0.906
choice fold 2 test accuracy: 0.772 auc: 0.818
choice fold 3 test accuracy: 0.842 auc: 0.880
choice fold 4 test accuracy: 0.860 auc: 0.963
choice fold 5 test accuracy: 0.825 auc: 0.791
choice mean of 5-fold cv accuracy: 0.825 auc: 0.872
choice sd of 5-fold cv accuracy: 0.029 auc: 0.062
unsorted decoding ...
choice fold 1 test accuracy: 0.897 auc: 0.950
choice fold 2 test accuracy: 0.789 auc: 0.892
choice fold 3 test accuracy: 0.947 auc: 0.801
choice fold 4 test accuracy: 0.860 auc: 0.935
choice fold 5 test accuracy: 0.842 auc: 0.849
choice mean of 5-fold cv accuracy: 0.867

In [99]:
# load decoding results for plotting
regional_decode_results = np.load(f'{rootpath}/{sub_id}/decode_results/{roi}_decode_results.npy', allow_pickle=True).item()
regional_tpca_decode_results = np.load(f'{rootpath}/{sub_id}/decode_results/{roi}_tpca_decode_results.npy', allow_pickle=True).item()

plotting

In [ ]:
# box plots for choice 



In [ ]:
# box plots for stimuli



In [ ]:
# visualize stimuli traces


